In [1]:
import itertools

import networkx as nx
import numpy as np
import pandas as pd
from disjoint_set import DisjointSet
from mbi import FactoredInference, Dataset, Domain
from scipy import sparse
from scipy.special import logsumexp
from sklearn.preprocessing import OrdinalEncoder

from cdp2adp import cdp_rho

c:\users\wildeh\src\private-pgm\src\mbi\__init__.py:15: UserWarning: MixtureInference disabled, please install jax and jaxlib
  warnings.warn('MixtureInference disabled, please install jax and jaxlib')


In [2]:
"""
Taken from https://github.com/ryan112358/private-pgm/blob/master/mechanisms/mst.py

This is a generalization of the winning mechanism from the 
2018 NIST Differential Privacy Synthetic Data Competition.

Unlike the original implementation, this one can work for any discrete dataset,
and does not rely on public provisional data for measurement selection.  
"""


def MST(data, epsilon, delta):
    rho = cdp_rho(epsilon, delta)
    sigma = np.sqrt(3 / (2 * rho))
    cliques = [(col,) for col in data.domain]
    log1 = measure(data, cliques, sigma)
    data, log1, undo_compress_fn = compress_domain(data, log1)
    cliques = select(data, rho / 3.0, log1)
    log2 = measure(data, cliques, sigma)
    engine = FactoredInference(data.domain, iters=5000)
    est = engine.estimate(log1 + log2)
    synth = est.synthetic_data()
    return undo_compress_fn(synth)


def measure(data, cliques, sigma, weights=None):
    if weights is None:
        weights = np.ones(len(cliques))
    weights = np.array(weights) / np.linalg.norm(weights)
    measurements = []
    for proj, wgt in zip(cliques, weights):
        x = data.project(proj).datavector()
        y = x + np.random.normal(loc=0, scale=sigma / wgt, size=x.size)
        Q = sparse.eye(x.size)
        measurements.append((Q, y, sigma / wgt, proj))
    return measurements


def compress_domain(data, measurements):
    supports = {}
    new_measurements = []
    for Q, y, sigma, proj in measurements:
        col = proj[0]
        sup = y >= 3 * sigma
        supports[col] = sup
        if supports[col].sum() == y.size:
            new_measurements.append((Q, y, sigma, proj))
        else:  # need to re-express measurement over the new domain
            y2 = np.append(y[sup], y[~sup].sum())
            I2 = np.ones(y2.size)
            I2[-1] = 1.0 / np.sqrt(y.size - y2.size + 1.0)
            y2[-1] /= np.sqrt(y.size - y2.size + 1.0)
            I2 = sparse.diags(I2)
            new_measurements.append((I2, y2, sigma, proj))
    undo_compress_fn = lambda data: reverse_data(data, supports)
    return transform_data(data, supports), new_measurements, undo_compress_fn


def exponential_mechanism(q, eps, sensitivity, prng=np.random, monotonic=False):
    coef = 1.0 if monotonic else 0.5
    scores = coef * eps / sensitivity * q
    probas = np.exp(scores - logsumexp(scores))
    return prng.choice(q.size, p=probas)


def select(data, rho, measurement_log, cliques=[]):
    engine = FactoredInference(data.domain, iters=1000)
    est = engine.estimate(measurement_log)

    weights = {}
    candidates = list(itertools.combinations(data.domain.attrs, 2))
    for a, b in candidates:
        xhat = est.project([a, b]).datavector()
        x = data.project([a, b]).datavector()
        weights[a, b] = np.linalg.norm(x - xhat, 1)

    T = nx.Graph()
    T.add_nodes_from(data.domain.attrs)
    ds = DisjointSet()

    for e in cliques:
        T.add_edge(*e)
        ds.union(*e)

    r = len(list(nx.connected_components(T)))
    epsilon = np.sqrt(8 * rho / (r - 1))
    for i in range(r - 1):
        candidates = [e for e in candidates if not ds.connected(*e)]
        wgts = np.array([weights[e] for e in candidates])
        idx = exponential_mechanism(wgts, epsilon, sensitivity=1.0)
        e = candidates[idx]
        T.add_edge(*e)
        ds.union(*e)

    return list(T.edges)


def transform_data(data, supports):
    df = data.df.copy()
    newdom = {}
    for col in data.domain:
        support = supports[col]
        size = support.sum()
        newdom[col] = int(size)
        if size < support.size:
            newdom[col] += 1
        mapping = {}
        idx = 0
        for i in range(support.size):
            mapping[i] = size
            if support[i]:
                mapping[i] = idx
                idx += 1
        assert idx == size
        df[col] = df[col].map(mapping)
    newdom = Domain.fromdict(newdom)
    return Dataset(df, newdom)


def reverse_data(data, supports):
    df = data.df.copy()
    newdom = {}
    for col in data.domain:
        support = supports[col]
        mx = support.sum()
        newdom[col] = int(support.size)
        idx, extra = np.where(support)[0], np.where(~support)[0]
        mask = df[col] == mx
        if extra.size == 0:
            pass
        else:
            df.loc[mask, col] = np.random.choice(extra, mask.sum())
        df.loc[~mask, col] = idx[df.loc[~mask, col]]
    newdom = Domain.fromdict(newdom)
    return Dataset(df, newdom)

In [3]:
def encode_data(data):
    """Use an ordinal encoder to convert categorical data."""

    encoder = OrdinalEncoder().fit(data)
    data = pd.DataFrame(
        encoder.transform(data).astype(int), columns=data.columns
    )

    return data, encoder


def load_data(path, skip=True):
    """Read in and prepare the data for synthesis."""

    if skip is True:
        data = pd.read_csv(path, skiprows=1, header=False)
    
    data = pd.read_csv(path, skiprows=1).drop("Person ID", axis=1)
    data, encoder = encode_data(data)

    return data, encoder
    

def decode_data(data, encoder):
    """Decode the data so it is interpretable."""

    return pd.DataFrame(encoder.inverse_transform(data), columns=data.columns)


def get_sparse_column_pair(data):
    """Get the column pair with the lowest two-way marginal count."""

    pair = None
    min_cell_count = np.inf
    for a, b in itertools.combinations(data.columns, r=2):
        count = data.groupby([a, b]).size().min()
        if count < min_cell_count:
            min_cell_count = count
            pair = [a, b]

    return pair


def calculate_marginal_table(data, by, encoder=None):
    """Get a marginal table for a column or set thereof."""

    if encoder is not None:
        data = data.pipe(decode_data, encoder)

    table = data.groupby(by).size().rename("count").reset_index()

    if not isinstance(by, str) and len(by) > 1:
        table = table.pivot(*by, "count")

    return table

In [4]:
data, encoder = load_data("main.csv")
a, b = pair = get_sparse_column_pair(data)

np.random.seed(0)
epsilon, delta = 1, 10 ** -np.ceil(np.log10(len(data)))
dataset = Dataset(data, Domain.fromdict(data.nunique().to_dict()))

KeyError: "['Person ID'] not found in axis"

In [ ]:
rho = cdp_rho(epsilon, delta)
sigma = np.sqrt(3 / (2 * rho))
cliques = [(col,) for col in dataset.domain]
log1 = measure(dataset, cliques, sigma)
dataset, log1, undo_compress_fn = compress_domain(dataset, log1)

cliques = select(dataset, rho / 3.0, log1, cliques=[tuple(pair)])
log2 = measure(dataset, cliques, sigma)

engine = FactoredInference(dataset.domain, iters=5000)
est = engine.estimate(log1 + log2)
synth = est.synthetic_data()

## Data preview

In [ ]:
data.pipe(decode_data, encoder).sample(5, random_state=0)

In [ ]:
synth.df.pipe(decode_data, encoder).head()

## One-way marginals

In [ ]:
def get_noisy_one_way(log1, col):
    """Get the noisy marginal table for a column."""

    noisy = (
        pd.DataFrame(
            {
                "noisy_count": next(
                    counts for _, counts, _, clique in log1 if clique == (col,)
                )
            }
        )
        .rename_axis(index=col)
        .reset_index()
    )

    return noisy


def compare_one_ways(data, log1, synth, col):
    """Get a table comparing all the various one-way tables."""

    original = data.pipe(calculate_marginal_table, col).rename(
        {"count": "observed_count"}, axis=1
    )
    noisy = get_noisy_one_way(log1, col)
    synthetic = synth.df.pipe(calculate_marginal_table, col).rename(
        {"count": "synthetic_count"}, axis=1
    )

    comparison = (
        original.merge(noisy, on=col).merge(synthetic, on=col).set_index(col)
    )
    comparison.index += 1

    return comparison

In [ ]:
compare_one_ways(data, log1, synth, a)

In [ ]:
compare_one_ways(data, log1, synth, b)

## Two-way marginals

In [ ]:
def get_noisy_two_way(log2, pair, domain):
    """Get the noisy marginal table for a pair of columns."""

    a, b = pair
    shape = domain.project(pair).shape

    noisy = pd.DataFrame(
        next(
            counts for _, counts, _, clique in log2 if clique == tuple(pair)
        ).reshape(shape)
    ).rename_axis(index=a, columns=b)

    noisy.index += 1
    noisy.columns += 1
    
    return noisy

In [ ]:
calculate_marginal_table(data, pair, encoder)

In [ ]:
get_noisy_two_way(log2, pair, dataset.domain)

In [ ]:
calculate_marginal_table(synth.df, pair, encoder)